In [1]:
import sys
import os

sys.path.append(os.path.abspath("app"))

In [2]:
UPLOAD_FOLDER = 'video'
RESULT_FOLDER = 'results'

In [3]:
import cv2
from detector import ObjectDetector
from tracker import ObjectTracker
from counter import RegionCounter
from speed_estimator import SpeedEstimator
from utils import draw_detections, draw_regions, draw_speed

def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    detector = ObjectDetector()
    tracker = ObjectTracker()
    regions = [('Entry', (100, 100, 300, 300)), ('Exit', (400, 100, 600, 300))]
    counter = RegionCounter(regions)
    speed_estimator = SpeedEstimator(fps=fps, ppm=10)  # PPM: pixels per meter

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        detections = detector.detect(frame)  # list of (x, y, w, h)
        tracked_objects = tracker.update(frame, detections)  # list of (id, (x, y, w, h))

        # Update counter and speed estimator
        for obj_id, bbox in tracked_objects:
            counter.update(obj_id, bbox)
            speed = speed_estimator.update(obj_id, bbox)
            draw_speed(frame, obj_id, bbox, speed)

        draw_regions(frame, regions)
        draw_detections(frame, tracked_objects)  # now supports drawing with IDs
        out.write(frame)

    cap.release()
    out.release()

In [4]:
process_video("videos/Car_Road_Video_for_OD.mov", "results/Process_Video.mov")


0: 384x640 7 cars, 2 buss, 1 train, 64.5ms
Speed: 5.2ms preprocess, 64.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'init'
> Overload resolution failed:
>  - Can't parse 'boundingBox'. Input argument doesn't provide sequence protocol
>  - Can't parse 'boundingBox'. Input argument doesn't provide sequence protocol
